# Numpy vectorizing

In [17]:
import numpy as np

In [18]:
def root(a, b, c):
    D = b ** 2 - 4 * a * c
    if D < 0:
        return np.nan, np.nan
    x1 = (-b + np.sqrt(D)) / (2 * a)
    x2 = (-b - np.sqrt(D)) / (2 * a)
    return x1, x2
root(1, 3, 4), root(-1, 3, 4), root(1, 0, 0)

((nan, nan), (-1.0, 4.0), (0.0, 0.0))

In [19]:
a = np.random.randn(int(1e6))
b = np.random.randn(int(1e6))
c = np.random.randn(int(1e6))
root(a, b, c)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [20]:
%%time
np.array([root(av, bv, cv) for av, bv, cv in zip(a, b, c)])

CPU times: user 3.64 s, sys: 65.9 ms, total: 3.71 s
Wall time: 3.71 s


array([[-0.55797188,  1.11121928],
       [        nan,         nan],
       [ 0.76166857, -1.70637551],
       ...,
       [-0.56859783,  0.98659831],
       [        nan,         nan],
       [-0.53531175,  0.18339357]])

In [21]:
root_vec = np.vectorize(root)
np.vectorize?

In [22]:
%%time
root_vec(a, b, c)

CPU times: user 1.85 s, sys: 71.9 ms, total: 1.92 s
Wall time: 1.92 s


(array([-0.55797188,         nan,  0.76166857, ..., -0.56859783,
                nan, -0.53531175]),
 array([ 1.11121928,         nan, -1.70637551, ...,  0.98659831,
                nan,  0.18339357]))

In [23]:
%%time
root_vec(a, b, 7)

CPU times: user 1.66 s, sys: 62.7 ms, total: 1.73 s
Wall time: 1.73 s


(array([-1.72246968, -2.63787563,         nan, ..., -2.20722406,
                nan, -1.84415698]),
 array([2.27571708, 2.04353033,        nan, ..., 2.62522454,        nan,
        1.4922388 ]))

# Numba

In [24]:
import numba

In [25]:
root_jit = numba.jit(root)

In [26]:
%timeit root(23, 78, 19.0)

3.08 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
%timeit root_jit(23, 78, 19.0)

The slowest run took 11.09 times longer than the fastest. This could mean that an intermediate result is being cached.
786 ns ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%time
root_jit(a, b, 7)

SystemError: CPUDispatcher(<function root at 0x114035620>) returned a result with an error set

In [29]:
root_jit_vec = np.vectorize(root_jit)
%time root_jit_vec(a, b, 7)

CPU times: user 406 ms, sys: 57.2 ms, total: 464 ms
Wall time: 464 ms


(array([-1.72246968, -2.63787563,         nan, ..., -2.20722406,
                nan, -1.84415698]),
 array([2.27571708, 2.04353033,        nan, ..., 2.62522454,        nan,
        1.4922388 ]))

In [30]:
@np.vectorize
@numba.jit
def fast_root(a, b, c):
    D = b ** 2 - 4 * a * c
    if D < 0:
        return np.nan, np.nan
    x1 = (-b + np.sqrt(D)) / (2 * a)
    x2 = (-b - np.sqrt(D)) / (2 * a)
    return x1, x2
%time fast_root(a, b, 7)

CPU times: user 391 ms, sys: 58.2 ms, total: 449 ms
Wall time: 448 ms


(array([-1.72246968, -2.63787563,         nan, ..., -2.20722406,
                nan, -1.84415698]),
 array([2.27571708, 2.04353033,        nan, ..., 2.62522454,        nan,
        1.4922388 ]))

In [31]:
@numba.vectorize
def fast_root(a, b, c):
    D = b ** 2 - 4 * a * c
    if D < 0:
        return np.nan, np.nan
    x1 = (-b + np.sqrt(D)) / (2 * a)
    x2 = (-b - np.sqrt(D)) / (2 * a)
    return x1, x2
%time fast_root(a, b, 7)

NotImplementedError: (float64 x 2) cannot be represented as a Numpy dtype

# But run the profiler first

In [32]:
def fib(n):
    if n <= 1:
        return 1
    else:
        return fib(n-2) + fib(n-1)
%time fib(33)

CPU times: user 1.56 s, sys: 3.5 ms, total: 1.56 s
Wall time: 1.56 s


5702887

In [35]:
@numba.jit
def fib_jit(n):
    if n <= 1:
        return 1
    else:
        return fib_jit(n-2) + fib_jit(n-1)
%time fib_jit(33)

CPU times: user 63.4 ms, sys: 1.79 ms, total: 65.2 ms
Wall time: 64.2 ms


5702887

In [34]:
%time fib_jit(41)

CPU times: user 1.28 s, sys: 5.26 ms, total: 1.28 s
Wall time: 1.28 s


267914296

In [103]:
%prun fib(33)

In [104]:
%prun fib_jit(41)

In [112]:
from functools import lru_cache
@lru_cache(None)
def fib_cache(n):
    if n <= 1:
        return 1
    else:
        return fib_cache(n-2) + fib_cache(n-1)
%time fib_cache(33)

CPU times: user 20 µs, sys: 1 µs, total: 21 µs
Wall time: 24.1 µs


In [ ]:
lru_cache?

In [111]:
%time fib_cache(500)

CPU times: user 261 µs, sys: 1e+03 ns, total: 262 µs
Wall time: 267 µs


225591516161936330872512695036072072046011324913758190588638866418474627738686883405015987052796968498626

# Optimizing numpy algebra

Multiplying/adding arrays incurs a lot of overhead, because lots of intermediate arrays get created and discarded again

In [48]:
a = np.random.randn(int(1e7))
b = np.random.randn(int(1e7))
%timeit a * b + 4.1 * a < 3 * b

139 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
import numexpr as ne
%timeit ne.evaluate('a * b + 4.1 * a < 3 * b')

11.4 ms ± 484 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Efficiency gain reduces once the operations become more complex

In [54]:
%timeit a * np.sqrt(abs(b)) < 3 * b

141 ms ± 1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
%timeit ne.evaluate('a * sqrt(abs(b)) < 3 * b')

12.6 ms ± 986 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Cython

http://docs.cython.org/en/latest/

Compiles most python programs to C code for extra speed (but less introspection). Also allows direct calling of C library code.

Can include python classes and much more.

In [69]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [70]:
def primes(kmax):
    p = {}
    result = []
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
CPU times: user 67.4 ms, sys: 1.22 ms, total: 68.6 ms
Wall time: 68 ms


In [91]:
%%cython -a
def cprimes(int kmax):
    cdef int[1000] p
    result = []
    if kmax > 1000:
        kmax = 1000
    cdef int k, n, i
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [92]:
print(cprimes(10))
%time _ = cprimes(10000)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
CPU times: user 2.2 ms, sys: 10 µs, total: 2.21 ms
Wall time: 2.22 ms


## Defining C-style function

In [47]:
#%%cython -a
def fib(n):
    if n <= 1:
        return 1
    else:
        return fib(n-2) + fib(n-1)

In [48]:
%time fib(33)

CPU times: user 1.48 s, sys: 1.79 ms, total: 1.48 s
Wall time: 1.48 s


5702887

In [49]:
%time fib_jit(33)

CPU times: user 27.5 ms, sys: 73 µs, total: 27.5 ms
Wall time: 27.5 ms


5702887

In [56]:
%%cython -a
cdef int cfib(int n) nogil:
    if n <= 1:
        return 1
    else:
        return cfib(n-2) + cfib(n-1)

def fib(n):
    return cfib(n)

In [57]:
%time fib(33)

CPU times: user 10.2 ms, sys: 138 µs, total: 10.4 ms
Wall time: 10.4 ms


5702887

## Cython in production code

In [ ]:
def root(a, b, c):
    D = b ** 2 - 4 * a * c
    if D < 0:
        return NAN, NAN
    x1 = (-b + sqrt(D)) / (2 * a)
    x2 = (-b - sqrt(D)) / (2 * a)
    return x1, x2

In [68]:
%time np.vectorize(root)(a, b, 7)

CPU times: user 236 ms, sys: 53.7 ms, total: 290 ms
Wall time: 288 ms


(array([-1.72246969, -2.6378758 ,         nan, ..., -2.20722389,
                nan, -1.84415698]),
 array([2.27571702, 2.04353046,        nan, ..., 2.62522459,        nan,
        1.49223876]))

In [66]:
%time root_jit_vec(a, b, 7)

CPU times: user 353 ms, sys: 57.8 ms, total: 411 ms
Wall time: 410 ms


(array([-1.72246968, -2.63787563,         nan, ..., -2.20722406,
                nan, -1.84415698]),
 array([2.27571708, 2.04353033,        nan, ..., 2.62522454,        nan,
        1.4922388 ]))

In [ ]:
%%cython -a
from libc.math cimport sqrt, NAN

def root(float a, float b, float c):
    cdef float D, x1, x2
    D = b ** 2 - 4 * a * c
    if D < 0:
        return NAN, NAN
    x1 = (-b + sqrt(D)) / (2 * a)
    x2 = (-b - sqrt(D)) / (2 * a)
    return x1, x2
print(root(1, 3, 4), root(-1, 3, 4), root(1, 0, 0))

## Cython in production code

http://docs.cython.org/en/latest/src/tutorial/cython_tutorial.html

# Steps to optimizing code:
- Profile where the bottleneck is (%prun)
- Is there a faster algorithm for the bottleneck?
- If the bottleneck is vectorized: can we optimize with numexpr?
- If the internal part of the loop can not be vectorized:
    - numba jit if simple enough otherwise cython
    - the loop itself can be sped up with numpy.vectorize
- If the bottleneck gets called too often: cache the result
- If even that fails: pycuda
- If that is too slow: learn to optimize cuda code or find an easier problem